# Importing Deploy

In [ ]:
%pip install joblib
%pip install --upgrade scikit-learn
%pip install openpyxl
%pip install Flask-CORS
%pip install flask-ngrok

from flask import Flask, jsonify, request
from flask_cors import CORS
from flask_ngrok import run_with_ngrok
import joblib
from joblib import load
import pandas as pd
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

# Substitua 'caminho/do/seu/arquivo.csv' pelo caminho real do seu arquivo
caminho_arquivo = r'C:\Users\Inteli\Documents\GitHub\grupo2\web\tabelas\df_final (1).csv'

# Leia o arquivo CSV
df = pd.read_csv(caminho_arquivo)

caminho_modelo = r'C:\Users\Inteli\Documents\GitHub\grupo2\web\tabelas\RandomForestModel.joblib'

# Carregue o modelo
modelo = joblib.load(caminho_modelo)

df_final = df
rf_loaded = modelo

df_final.drop('Unnamed: 0', axis=1, inplace=True)

# Caminho para o arquivo Excel
caminho_excel = r'C:\Users\Inteli\Documents\GitHub\grupo2\web\tabelas\Predict_Churn (1).xlsx'

# Carregar a tabela do Excel
df_excel = pd.read_excel(caminho_excel)

df_novo = pd.DataFrame()

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

@app.route('/api/items', methods=['GET'])
def get_items():
    # Converter o DataFrame para um dicionário
    data = df_excel.to_dict(orient='records')
    
    print(jsonify(data), "bbbbbbbbb")
    return jsonify(data)


@app.route('/api/predict', methods=['POST', 'GET'])
def predict():
    global df_novo

    df_novo = pd.DataFrame()

    # Recebe os dados do frontend
    data = request.get_json()

    colunas = ['cons_12m', 'cons_gas_12m', 'cons_last_month',
    'forecast_base_bill_ele', 'forecast_cons_12m', 'forecast_cons_year',
    'forecast_discount_energy', 'forecast_meter_rent_12m',
    'forecast_price_energy_p1', 'forecast_price_energy_p2',
    'forecast_price_pow_p1', 'has_gas', 'imp_cons', 'margin_gross_pow_ele',
    'margin_net_pow_ele', 'nb_prod_act', 'net_margin', 'num_years_antig',
    'pow_max', 'channel_sales_epumfxlbckeskwekxbiuasklxalciiuu',
    'channel_sales_ewpakwlliwisiwduibdlfmalxowmwpci',
    'channel_sales_fixdbufsefwooaasfcxdxadsiekoceaa',
    'channel_sales_foosdfpfkusacimwkcsosbicdxkicaua',
    'channel_sales_lmkebamcaaclubfxadlmueccxoimlema',
    'channel_sales_no_fill',
    'channel_sales_sddiedcslfslkckwlfkdpoeeailfpeds',
    'channel_sales_usilxuppasemubllopkaafesmlibmsdf',
    'origin_up_ewxeelcelemmiwuafmddpobolfuxioce',
    'origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws',
    'origin_up_ldkssxwpmemidmecebumciepifcamkci',
    'origin_up_lxidpiddsbxsbosboudacockeimpuepw', 'origin_up_no_fill',
    'origin_up_usapbepcfoloekilkwsdiboslwaxobdp', 'date_end_sin',
    'date_end_cos', 'date_activ_sin', 'dia_activ_cos',
    'date_modif_prod_sin', 'date_modif_prod_cos', 'date_renewal_sin',
    'date_renewal_cos', 'price_p1_var', 'price_p2_var', 'price_p3_var',
    'price_p1_fix', 'price_p2_fix', 'price_p3_fix', 'price_var_mean',
    'price_fix_mean', 'price_var', 'price_fix', 'consumption']

    df_novo = pd.concat([df_novo, pd.DataFrame([data])], ignore_index=True)

    print("Dados recebidos do frontend:")
    print(df_novo)

    id = df_novo.loc[0, 'id']

    linha_selecionada = df_final.loc[df_final['id'] == id]

    colunas_a_adicionar = [coluna for coluna in linha_selecionada.columns if coluna not in df_novo.columns]

    for coluna in colunas_a_adicionar:
        df_novo[coluna] = linha_selecionada[coluna].values[0]

    X_novo = df_novo.drop(columns=['id', 'churn'])
    X_novo = X_novo[colunas]

    new_previsionRF = rf_loaded.predict_proba(X_novo)

    new_prob_predict = []

    for i in range(len(new_previsionRF)):
        new_prob_predict.append(new_previsionRF[i][1])

    df_prob_novo = pd.DataFrame(new_prob_predict)

    x_novo = pd.DataFrame(X_novo)

    id_col_nova = pd.DataFrame(df_novo['id'])

    df_final_novo = pd.concat([id_col_nova, x_novo, df_prob_novo], axis=1)

    df_final_novo['Probabilidade Churn'] = df_prob_novo

    df_final_novo = df_final_novo[['id', 'net_margin', 'price_p1_var', 'price_p1_fix', 'num_years_antig', 'cons_12m', 'consumption', 'Probabilidade Churn']]

    resultado = df_final_novo.merge(df_final, on='id', how='inner')

    print(resultado)

    if resultado.empty:
        return jsonify({'error': 'Sem resultados'})

    if 'net_margin_x' not in resultado.columns:
        return jsonify({'error': 'A coluna net_margin_x não está presente'})

    id_resultado = resultado.iloc[0]['id']
    net_margin_resultado = resultado.iloc[0]['net_margin_x']
    price_p1_var_resultado = resultado.iloc[0]['price_p1_var_x']
    price_p1_fix_resultado = resultado.iloc[0]['price_p1_fix_x']
    num_years_antig_resultado = resultado.iloc[0]['num_years_antig_x']
    cons_12m_resultado = resultado.iloc[0]['cons_12m_x']
    consumption_resultado = resultado.iloc[0]['consumption_x']
    probability_resultado = resultado.iloc[0]['Probabilidade Churn']

    data = {
        'id': id_resultado,
        'net_margin': net_margin_resultado,
        'price_p1_var': price_p1_var_resultado,
        'price_p1_fix': price_p1_fix_resultado,
        'num_years_antig': num_years_antig_resultado,
        'cons_12m': cons_12m_resultado,
        'consumption': consumption_resultado,
        'churn': probability_resultado
    }

    print(jsonify(data), 'aaaaaaaaaaaa');

    return jsonify(data)

if __name__ == '__main__':
    app.run()
